In [136]:
import pandas as pd
import numpy as np

In [96]:
companies = pd.read_csv("companies.csv")
companies.shape

(66368, 10)

In [97]:
rounds2 = pd.read_csv("rounds2.csv")
rounds2.shape

(114949, 6)

In [98]:
mapping = pd.read_csv("mapping.csv")
mapping.shape

(688, 10)

In [99]:
companies.permalink = companies.permalink.str.upper()
rounds2.company_permalink = rounds2.company_permalink.str.upper()

In [100]:
distinct_companies = companies.permalink.unique()
distinct_companies_r = rounds2.company_permalink.unique()

In [101]:
z = companies.permalink.isin(rounds2.company_permalink)

p = rounds2.company_permalink.isin(companies.permalink)

In [102]:
master_frame = rounds2.merge(companies, left_on ='company_permalink', right_on = 'permalink',how = 'outer')
master_frame.shape

(114949, 16)

In [103]:
master_frame.funding_round_type = master_frame.funding_round_type.str.lower()
funding_type_list = ['venture','seed','angel','private_equity']
funding_type_df = master_frame[master_frame['funding_round_type'].isin(funding_type_list)]
funding_type_df.shape

(94397, 16)

In [104]:
funding_type_group = funding_type_df.groupby(['funding_round_type'])
funding_type_group[['raised_amount_usd']].mean()

,raised_amount_usd
funding_round_type,
angel,9.586945e+05
private_equity,7.330859e+07
seed,7.198180e+05
venture,1.174895e+07


In [105]:
funding_type_mean = funding_type_df.groupby('funding_round_type').agg({'raised_amount_usd':'mean'})
funding_type_mean = funding_type_mean.sort_values('raised_amount_usd',ascending=False)
funding_type_mean.query('raised_amount_usd >= 5000000 & raised_amount_usd <= 15000000')

,raised_amount_usd
funding_round_type,
venture,1.174895e+07


In [213]:
master_frame_venture = master_frame[master_frame['funding_round_type'] == 'venture']
master_frame_venture.groupby('country_code').agg({'raised_amount_usd':'sum'}).sort_values(
                                        'raised_amount_usd',ascending=False).head(9)


,raised_amount_usd
country_code,
USA,4.225108e+11
CHN,3.983542e+10
GBR,2.024563e+10
IND,1.439186e+10
CAN,9.583332e+09
FRA,7.259537e+09
ISR,6.907515e+09
DEU,6.346960e+09
JPN,3.363677e+09


In [107]:
mapping = mapping.dropna(thresh=10)
mapping.head(6)

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1
5,Active Lifestyle,0,0,0,0,1,0,0,0,0
6,Ad Targeting,0,0,0,0,0,0,0,0,1


In [108]:
mapping.category_list = mapping.category_list.str.replace('0','na')
mapping.category_list = mapping.category_list.str.replace('2.na','2.0')

In [109]:
mapping = pd.melt(mapping,id_vars=['category_list'])
mapping.head(10)

,category_list,variable,value
0,3D,Automotive & Sports,0
1,3D Printing,Automotive & Sports,0
2,3D Technology,Automotive & Sports,0
3,Accounting,Automotive & Sports,0
4,Active Lifestyle,Automotive & Sports,0
5,Ad Targeting,Automotive & Sports,0
6,Advanced Materials,Automotive & Sports,0
7,Adventure Travel,Automotive & Sports,1
8,Advertising,Automotive & Sports,0
9,Advertising Exchanges,Automotive & Sports,0


In [110]:
mapping = mapping[mapping['value'] == 1]
del mapping['value']

In [111]:
mapping.shape
master_frame.head(5)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/ORGANIZATION/-FAME,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,5/1/15,10000000.0,/ORGANIZATION/-FAME,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,4/9/14
2,/ORGANIZATION/-QOUNTER,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,1/3/14,700000.0,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,4/9/14
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
4,/ORGANIZATION/0-6-COM,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/ORGANIZATION/0-6-COM,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,1/1/07


In [112]:
master_frame['primary_sector'] = master_frame.category_list.str.split('|',1).str[0]
master_frame.head(5)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,primary_sector
0,/ORGANIZATION/-FAME,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,5/1/15,10000000.0,/ORGANIZATION/-FAME,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,Media
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,4/9/14,Application Platforms
2,/ORGANIZATION/-QOUNTER,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,1/3/14,700000.0,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,4/9/14,Application Platforms
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,Apps
4,/ORGANIZATION/0-6-COM,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/ORGANIZATION/0-6-COM,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,1/1/07,Curated Web


In [113]:
master_frame['primary_sector'] = master_frame['primary_sector'].str.lower()
mapping['category_list'] = mapping['category_list'].str.lower()

In [214]:
master_frame = master_frame.merge(mapping, left_on ='primary_sector', 
                                  right_on = 'category_list',how = 'left')

master_frame.head(5)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,...,country_code,state_code,region,city,founded_at,primary_sector,category_list_y,variable_x,category_list,variable_y
0,/ORGANIZATION/-FAME,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,5/1/15,10000000.0,/ORGANIZATION/-FAME,#fame,http://livfame.com,Media,...,IND,16,Mumbai,Mumbai,NaN,media,media,Entertainment,media,Entertainment
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,...,USA,DE,DE - Other,Delaware City,4/9/14,application platforms,application platforms,"News, Search and Messaging",application platforms,"News, Search and Messaging"
2,/ORGANIZATION/-QOUNTER,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,1/3/14,700000.0,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,...,USA,DE,DE - Other,Delaware City,4/9/14,application platforms,application platforms,"News, Search and Messaging",application platforms,"News, Search and Messaging"
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,...,NaN,NaN,NaN,NaN,NaN,apps,apps,"News, Search and Messaging",apps,"News, Search and Messaging"
4,/ORGANIZATION/0-6-COM,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/ORGANIZATION/0-6-COM,0-6.com,http://www.0-6.com,Curated Web,...,CHN,22,Beijing,Beijing,1/1/07,curated web,curated web,"News, Search and Messaging",curated web,"News, Search and Messaging"


In [115]:
master_frame.shape

(114949, 19)

In [116]:
D1 = master_frame.query('funding_round_type == "venture" & country_code == "USA" & raised_amount_usd >=5000000 & raised_amount_usd <= 15000000')
D1.shape

(12150, 19)

In [118]:
D2 = master_frame.query('funding_round_type == "venture" & country_code == "GBR" & raised_amount_usd >=5000000 & raised_amount_usd <= 15000000')
D2.shape

(628, 19)

In [119]:
D3 = master_frame.query('funding_round_type == "venture" & country_code == "IND" & raised_amount_usd >=5000000 & raised_amount_usd <= 15000000')
D3.shape

(330, 19)

D1_top3 = D1.groupby('variable').agg({'raised_amount_usd':'count'})
D1_top3

In [170]:
def my_agg(x):
    names = {
        'investment_sector_count': x['raised_amount_usd'].count(),
        'investment_sector_total':  x['raised_amount_usd'].mean()}
    return pd.Series(names, index=['investment_sector_count', 'investment_sector_total'])

In [172]:
D1_group_df = D1.groupby('variable').apply(my_agg)
D1 = D1.merge(D1_group_df,left_on = 'variable', right_on = 'variable',how='left')

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,...,country_code,state_code,region,city,founded_at,primary_sector,category_list_y,variable,investment_sector_count,investment_sector_total
0,/ORGANIZATION/0XDATA,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,A,19-07-2014,8900000.0,/ORGANIZATION/0XDATA,H2O.ai,http://h2o.ai/,Analytics,...,USA,CA,SF Bay Area,Mountain View,1/1/11,analytics,analytics,"Social, Finance, Analytics, Advertising",2714.0,8.772062e+06
1,/ORGANIZATION/1-MAINSTREAM,/funding-round/b952cbaf401f310927430c97b68162ea,venture,NaN,17-03-2015,5000000.0,/ORGANIZATION/1-MAINSTREAM,1 Mainstream,http://www.1mainstream.com,Apps|Cable|Distribution|Software,...,USA,CA,SF Bay Area,Cupertino,1/3/12,apps,apps,"News, Search and Messaging",1583.0,8.826006e+06
2,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,NaN,16-12-2014,11999347.0,/ORGANIZATION/128-TECHNOLOGY,128 Technology,http://www.128technology.com/,Service Providers|Technology,...,USA,MA,Boston,Burlington,7/7/14,service providers,service providers,Others,2950.0,8.922375e+06
3,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/424129ce1235cfab2655ee81305f7c2b,venture,C,15-10-2013,15000000.0,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,http://www.1366tech.com,Manufacturing,...,USA,MA,Boston,Bedford,1/1/07,manufacturing,manufacturing,Manufacturing,799.0,9.084547e+06
4,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/6d3f3797371956ece035b8478c1441b2,venture,C,9/4/15,5000000.0,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,http://www.1366tech.com,Manufacturing,...,USA,MA,Boston,Bedford,1/1/07,manufacturing,manufacturing,Manufacturing,799.0,9.084547e+06
5,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/786f61aa9866f4471151285f5c56be36,venture,A,3/2/10,5150000.0,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,http://www.1366tech.com,Manufacturing,...,USA,MA,Boston,Bedford,1/1/07,manufacturing,manufacturing,Manufacturing,799.0,9.084547e+06
6,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,A,27-03-2008,12400000.0,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,http://www.1366tech.com,Manufacturing,...,USA,MA,Boston,Bedford,1/1/07,manufacturing,manufacturing,Manufacturing,799.0,9.084547e+06
7,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/ab99fc5a53717b1b53fd6aa5687c5fa9,venture,B,16-12-2010,6000000.0,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,http://www.1366tech.com,Manufacturing,...,USA,MA,Boston,Bedford,1/1/07,manufacturing,manufacturing,Manufacturing,799.0,9.084547e+06
8,/ORGANIZATION/170-SYSTEMS,/funding-round/b84bb882ca873f5fb96535671981196d,venture,A,16-04-2002,14000000.0,/ORGANIZATION/170-SYSTEMS,170 Systems,http://www.170systems.com,Software,...,USA,MA,Boston,Bedford,1/1/90,software,software,Others,2950.0,8.922375e+06
9,/ORGANIZATION/17ZUOYE,/funding-round/69690484f51e15bc27ff52bfe472cd96,venture,A,1/1/11,5000000.0,/ORGANIZATION/17ZUOYE,17zuoye,http://www.17zuoye.com/,Education|Language Learning,...,USA,VA,VA - Other,Shanghai,1/1/07,education,education,Others,2950.0,8.922375e+06


In [215]:
D2_group_df = D2.groupby('variable').apply(my_agg)
D2 = D2.merge(D1_group_df,left_on = 'variable', right_on = 'variable',how='left')

In [216]:
D3_group_df = D3.groupby('variable').apply(my_agg)
D3 = D3.merge(D1_group_df,left_on = 'variable', right_on = 'variable',how='left')

In [187]:
D1_top3 = D1.groupby('variable').agg({'raised_amount_usd':'count'}).sort_values('raised_amount_usd',ascending=False).head(3)
D1_top3

(3, 1)

In [160]:


D2_top3 = D2.groupby('variable').agg({'raised_amount_usd':'count'}).sort_values('raised_amount_usd',ascending=False).head(3)
D2_top3

,raised_amount_usd
variable,
Others,147
"Social, Finance, Analytics, Advertising",133
Cleantech / Semiconductors,130


In [128]:
D3_top3 = D3.groupby('variable').agg({'raised_amount_usd':'count'}).sort_values('raised_amount_usd',ascending=False).head(3)
D3_top3

,raised_amount_usd
variable,
Others,110
"Social, Finance, Analytics, Advertising",60
"News, Search and Messaging",52


In [ ]:
Highest_investment_company <- function(df1,top3,n=1){
        df1[which(df1$main_sector==top3[["main_sector"]][n]),]%>%       
         group_by(company_permalink,name)%>%
                summarise_at(.vars="raised_amount_usd",
                             .fun=sum,na.rm = TRUE)%>%
                arrange(desc(raised_amount_usd))%>%
    
top_sector = D1_top3['raised_amunt_usd']

In [206]:
D1[D1.variable == D1_top3.index[0]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False)[:1]

,raised_amount_usd
name,
Virtustream,64300000.0


In [207]:
D1[D1.variable == D1_top3.index[1]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False)[:1]

,raised_amount_usd
name,
SST Inc. (Formerly ShotSpotter),67933006.0


In [208]:
D2[D2.variable == D2_top3.index[0]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False)[:1]

,raised_amount_usd
name,
Electric Cloud,37000000.0


In [209]:
D2[D2.variable == D2_top3.index[1]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False)[:1]

,raised_amount_usd
name,
Celltick Technologies,37500000.0


In [210]:
D3[D3.variable == D3_top3.index[0]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False)[:1]

,raised_amount_usd
name,
FirstCry.com,39000000.0


In [211]:
D3[D3.variable == D3_top3.index[1]].groupby('name').agg(
                                                {'raised_amount_usd':'sum'}).sort_values(
                                                'raised_amount_usd',ascending=False)[:1]

,raised_amount_usd
name,
Manthan Systems,50700000.0
